<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Scrape Dataset

In [16]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Use beautiful soup library to scrape the table from Wikipedia

In [17]:
from bs4 import BeautifulSoup
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List of postal codes of Canada: M', 'w') as fo:
    fo.write(article)

In [18]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List of postal codes of Canada: M').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')
#print(tables)
# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode','Borough', 'Neighborhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('List of postal codes of Canada: M.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighborhood = [td.text.strip() for td in tds[:3]]
        # Wikipedia does something funny with country names containing
        # accented characters: extract the correct string form.
        print('; '.join([postcode, borough, neighborhood]), file=fo)

In [19]:
#write the textfile to a Pandas dataframe
df = pd.read_csv('List of postal codes of Canada: M.txt', sep=';', 
                 converters={'Postcode': str, 'Borough': str, 'Neighborhood': str}, 
                 header=None, names=['Postcode','Borough', 'Neighborhood'])
print('Dimensions of the data frame is', df.shape)
df.head()

Dimensions of the data frame is (288, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Clean the dataset

Ignore cells with a borough that is Not assigned.

In [20]:
#Drop cells when Borough = Not assigned and show the final dimension of the array
df.drop(df[df.Borough == ' Not assigned'].index, inplace=True)
#reset the index to start from 0
df.index = np.arange(len(df))

Combine multiple rows into one row with the neighborhoods separated with a comma

Also, if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [21]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [22]:
df.Neighborhood[85] = "Queen's Park"
df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [23]:
print('Shape of the dataframe', df.shape)

Shape of the dataframe (103, 3)


 ## 2. Getting Geographical Data

Data is imported from the link provided in the assignment

In [24]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [25]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


asdasd

In [26]:
#add new columns to the data frame
df['Latitude'] = coordinates_df.Latitude
df['Longitude'] = coordinates_df.Longitude
df.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [27]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = df.drop(columns=['Neighborhood', 'Postcode','Borough'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

In [28]:
df.insert(0, 'Cluster Labels', kmeans.labels_)
df.head()

,Cluster Labels,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Lets get the location of Toronto

In [29]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters